# Data Warehouse Truven QA - Claim Diag

## Initialization

Just loading packages that will be used and initializing connection to GP DB.

In [1]:
import pandas as pd
import sys
import psycopg2
from tqdm import tqdm
sys.path.append('H:/uth_helpers')
from db_utils import get_dsn

In [2]:
connection = psycopg2.connect(get_dsn()+' keepalives=1 keepalives_idle=30 keepalives_interval=10')
connection.autocommit = True

## Row Count and Claim Count

Similar to the member_enrollment_monthly table, the claim_header table has the row count should equal the number of unique claims.

For this table, we extract claim data from the s, o, and f tables from the truven schema. 


In [52]:
query = ''' drop table if exists qa_reporting.dw_truv_claim_diag_counts;
create table qa_reporting.dw_truv_claim_diag_counts
(
    data_source bpchar(4),
    table_id_src text,
    calendar_year int,
    dw_row_count bigint,
    src_row_count bigint,
    row_count_diff bigint,
    row_count_percentage float,
    dw_uth_clm_id_count bigint,
    dw_src_clm_id_count bigint,
    src_clm_count bigint,
    clm_count_diff bigint,
    clm_count_percentage float,
    dw_uth_mbr_id_count bigint,
    dw_src_mbr_id_count bigint,
    src_mbr_count bigint,
    mbr_count_diff bigint,
    mbr_count_percentage float,
    date_generated date
);
'''

with connection.cursor() as cursor:
    cursor.execute(query)

In [53]:
with connection.cursor() as cursor:
    query = f'''
    insert into qa_reporting.dw_truv_claim_diag_counts
    (data_source, table_id_src, calendar_year, dw_row_count, dw_uth_clm_id_count, date_generated)
    select data_source, table_id_src, year, count(a.*), count(distinct uth_claim_id), current_date
    from (
        select distinct data_source, table_id_src, year, uth_claim_id, uth_member_id, diag_cd, icd_version
        from dw_staging.trum_claim_diag
        where diag_position = '1' or diag_position = 'P'
    ) a
    group by 1,2,3
    '''
    
    cursor.execute(query)

    query = f'''
    update qa_reporting.dw_truv_claim_diag_counts b
    set dw_src_clm_id_count = count
    from (
        select data_source, table_id_src, year, count(distinct claim_id_src) as count 
        from dw_staging.trum_claim_diag
        group by 1,2,3
    ) a
    where a.year = b.calendar_year
    and a.data_source = b.data_source
    and a.table_id_src = b.table_id_src
    '''

    cursor.execute(query)

    query = f'''
    update qa_reporting.dw_truv_claim_diag_counts b
    set dw_uth_mbr_id_count = count
    from (
        select data_source, table_id_src, year, count(distinct uth_member_id) as count 
        from dw_staging.trum_claim_diag
        group by 1,2,3
    ) a
    where a.year = b.calendar_year
    and a.data_source = b.data_source
    and a.table_id_src = b.table_id_src
    '''

    cursor.execute(query)
    
    query = f'''
    update qa_reporting.dw_truv_claim_diag_counts b
    set dw_src_mbr_id_count = count
    from (
        select data_source, table_id_src, year, count(distinct member_id_src) as count 
        from dw_staging.trum_claim_diag
        group by 1,2,3
    ) a
    where a.year = b.calendar_year
    and a.data_source = b.data_source
    and a.table_id_src = b.table_id_src
    '''

    cursor.execute(query)

In [54]:
with connection.cursor() as cursor:
    query = f'''
    insert into qa_reporting.dw_truv_claim_diag_counts
    (data_source, table_id_src, calendar_year, dw_row_count, dw_uth_clm_id_count, date_generated)
    select data_source, table_id_src, year, count(a.*), count(distinct uth_claim_id), current_date
    from (
        select distinct data_source, table_id_src, year, uth_claim_id, uth_member_id, diag_cd, icd_version
        from dw_staging.truc_claim_diag
        where diag_position = '1' or diag_position = 'P'
    ) a
    group by 1,2,3
    '''
    
    cursor.execute(query)

    query = f'''
    update qa_reporting.dw_truv_claim_diag_counts b
    set dw_src_clm_id_count = count
    from (
        select data_source, table_id_src, year,  count(distinct claim_id_src) as count 
        from dw_staging.truc_claim_diag
        group by 1,2,3
    ) a
    where a.year = b.calendar_year
    and a.data_source = b.data_source
    and a.table_id_src = b.table_id_src
    '''

    cursor.execute(query)

    query = f'''
    update qa_reporting.dw_truv_claim_diag_counts b
    set dw_uth_mbr_id_count = count
    from (
        select data_source, table_id_src, year, count(distinct uth_member_id) as count 
        from dw_staging.truc_claim_diag
        group by 1,2,3
    ) a
    where a.year = b.calendar_year
    and a.data_source = b.data_source
    and a.table_id_src = b.table_id_src
    '''

    cursor.execute(query)
    
    query = f'''
    update qa_reporting.dw_truv_claim_diag_counts b
    set dw_src_mbr_id_count = count
    from (
        select data_source, table_id_src, year, count(distinct member_id_src) as count 
        from dw_staging.truc_claim_diag
        group by 1,2,3
    ) a
    where a.year = b.calendar_year
    and a.data_source = b.data_source
    and a.table_id_src = b.table_id_src
    '''

    cursor.execute(query)

In [55]:
with connection.cursor() as cursor:
    query = '''update qa_reporting.dw_truv_claim_diag_counts a
    set src_row_count = b.row_count,
    row_count_diff = dw_row_count - b.row_count,
    row_count_percentage = 100. * abs( dw_row_count - b.row_count) / b.row_count,
    src_clm_count = clm_count,
    clm_count_diff = dw_uth_clm_id_count - b.clm_count,
    clm_count_percentage = 100. * abs(dw_uth_clm_id_count - b.clm_count) / b.clm_count,
    src_mbr_count = b.pat_count,
    mbr_count_diff = dw_uth_mbr_id_count - b.pat_count,
    mbr_count_percentage = 100. * abs(dw_uth_mbr_id_count - b.pat_count) / b.pat_count
    from qa_reporting.truven_counts b
    where a.calendar_year = b.year
    and a.table_id_src = b.table_name
    ;
    '''

    cursor.execute(query)

In [56]:
with connection.cursor() as cursor:
    query = '''
with truven_claims as (
        select 'ccaes' as table_source, year, enrolid, claim_id_derv
        from truven.ccaes
        where pdx is not null or dx1 is not null
        union 
        select 'mdcrs', year, enrolid, claim_id_derv
        from truven.mdcrs
        where pdx is not null or dx1 is not null
        union
        select 'ccaeo', year, enrolid, claim_id_derv
        from truven.ccaeo
        where dx1 is not null
        union
        select 'mdcro', year, enrolid, claim_id_derv
        from truven.mdcro
        where dx1 is not null
    ),
truven_claims_counts as (
        select table_source, year as calendar_year, count(*) as src_row_count, count(distinct enrolid) src_mbr_count, count(distinct claim_id_derv) src_clm_count
        from truven_claims
        group by 1,2
    )
update qa_reporting.dw_truv_claim_icd_proc_counts a
set src_row_count = b.src_row_count,
row_count_diff = a.dw_row_count - b.src_row_count,
row_count_percentage = 100. * abs(a.dw_row_count - b.src_row_count) / b.src_row_count,
src_clm_count = b.src_clm_count,
clm_count_diff = a.dw_uth_clm_id_count - b.src_clm_count,
clm_count_percentage = 100. * abs(a.dw_uth_clm_id_count - b.src_clm_count) / b.src_clm_count,
src_mbr_count = b.src_mbr_count,
mbr_count_diff = a.dw_uth_mbr_id_count - b.src_mbr_count,   
mbr_count_percentage = 100. * abs(a.dw_uth_mbr_id_count - b.src_mbr_count) / b.src_mbr_count
from truven_claims_counts b
where a.calendar_year = b.calendar_year
and a.table_id_src = b.table_source
;
    '''

    cursor.execute(query)

In [57]:
query = '''select * from qa_reporting.dw_truv_claim_diag_counts;'''

df = pd.read_sql(query, con=connection)

c:\Users\iperez9\.conda\envs\ip_env\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


Row counts may be very different due to how the data is transformed to match the Data Warehouse format.

In [58]:
df[df['data_source'] == 'trum'].sort_values(['table_id_src', 'calendar_year'])

,data_source,table_id_src,calendar_year,dw_row_count,src_row_count,row_count_diff,row_count_percentage,dw_uth_clm_id_count,dw_src_clm_id_count,src_clm_count,clm_count_diff,clm_count_percentage,dw_uth_mbr_id_count,dw_src_mbr_id_count,src_mbr_count,mbr_count_diff,mbr_count_percentage,date_generated
29,trum,mdcro,2011,106352416,234255047,-127902631,54.599733,106352416,106353876,106577308,-224892,0.211013,4743002,4743002,4750926,-7924,0.166789,2023-10-18
28,trum,mdcro,2012,99179814,229768218,-130588404,56.834842,99179814,99189188,99450749,-270935,0.272431,4383978,4383978,4391031,-7053,0.160623,2023-10-18
35,trum,mdcro,2013,87917218,208095580,-120178362,57.751521,87917218,87922903,88058549,-141331,0.160497,3805615,3805615,3812531,-6916,0.181402,2023-10-18
43,trum,mdcro,2014,79384198,193877526,-114493328,59.054461,79384198,79390350,79544615,-160417,0.201669,3505316,3505316,3514364,-9048,0.257458,2023-10-18
40,trum,mdcro,2015,49635408,121473554,-71838146,59.138918,49635408,49652846,49736837,-101429,0.203931,2032094,2032094,2034639,-2545,0.125084,2023-10-18
36,trum,mdcro,2016,48760927,118982284,-70221357,59.018330,48760927,48775710,48879522,-118595,0.242627,1969257,1969257,1972505,-3248,0.164664,2023-10-18
33,trum,mdcro,2017,34021662,81837912,-47816250,58.427994,34021662,34020344,34120595,-98933,0.289951,1362310,1362310,1364765,-2455,0.179884,2023-10-18
24,trum,mdcro,2018,26045222,59193949,-33148727,56.000195,26045222,26044371,26105522,-60300,0.230986,1056142,1056142,1057428,-1286,0.121616,2023-10-18
47,trum,mdcro,2019,42159886,99576131,-57416245,57.660651,42159886,42173028,42280035,-120149,0.284174,1541958,1541958,1543372,-1414,0.091618,2023-10-18
38,trum,mdcro,2020,39721411,94012150,-54290739,57.748641,39721411,39722867,39967420,-246009,0.615524,1595444,1595444,1597557,-2113,0.132264,2023-10-18


In [59]:
df[df['data_source'] == 'truc'].sort_values(['table_id_src', 'calendar_year'])

,data_source,table_id_src,calendar_year,dw_row_count,src_row_count,row_count_diff,row_count_percentage,dw_uth_clm_id_count,dw_src_clm_id_count,src_clm_count,clm_count_diff,clm_count_percentage,dw_uth_mbr_id_count,dw_src_mbr_id_count,src_mbr_count,mbr_count_diff,mbr_count_percentage,date_generated
4,truc,ccaeo,2011,464767402,1077661934,-612894532,56.872616,464767402,464815368,465722367,-954965,0.205050,42474852,42474852,42511345,-36493,0.085843,2023-10-18
5,truc,ccaeo,2012,474699272,1121961205,-647261933,57.690224,474699272,474746519,475937265,-1237993,0.260117,43260503,43260503,43288123,-27620,0.063805,2023-10-18
11,truc,ccaeo,2013,373488846,875696525,-502207679,57.349511,373488846,373506774,374452293,-963447,0.257295,33633095,33633095,33659811,-26716,0.079371,2023-10-18
19,truc,ccaeo,2014,398804173,952082337,-553278164,58.112428,398804173,398822102,399893229,-1089056,0.272337,36263937,36263937,36298645,-34708,0.095618,2023-10-18
17,truc,ccaeo,2015,253677591,599918568,-346240977,57.714663,253677591,253814030,254905571,-1227980,0.481739,22276330,22276330,22319003,-42673,0.191196,2023-10-18
12,truc,ccaeo,2016,262709234,623641018,-360931784,57.874927,262709234,262821831,264225295,-1516061,0.573776,22790094,22790094,22831148,-41054,0.179816,2023-10-18
8,truc,ccaeo,2017,243463013,573221660,-329758647,57.527248,243463013,243808815,245443871,-1980858,0.807051,20906587,20906587,20936299,-29712,0.141916,2023-10-18
1,truc,ccaeo,2018,250013677,589022269,-339008592,57.554461,250013677,250028329,251365523,-1351846,0.537801,21516356,21516356,21529034,-12678,0.058888,2023-10-18
23,truc,ccaeo,2019,241349179,566058850,-324709671,57.363235,241349179,241373783,242648619,-1299440,0.535523,20268922,20268922,20280269,-11347,0.055951,2023-10-18
14,truc,ccaeo,2020,216430506,485549961,-269119455,55.425698,216430506,216451366,217834256,-1403750,0.644412,18676658,18676658,18687285,-10627,0.056868,2023-10-18


## Diagnosis Codes

Here we will check if we have valid ICD Diagnosis codes using our reference tables.

In [60]:
query = '''drop table if exists qa_reporting.dw_truv_diag_counts;
select data_source, year, diag_cd, count(*) as diag_count
into qa_reporting.dw_truv_diag_counts
from dw_staging.trum_claim_diag
group by 1,2,3;

insert into qa_reporting.dw_truv_diag_counts
select data_source, year, diag_cd, count(*) as diag_count
from dw_staging.truc_claim_diag
group by 1,2,3;
'''

with connection.cursor() as cursor:
    cursor.execute(query)

In [61]:
diag_cd_df = pd.read_sql('select * from qa_reporting.dw_truv_diag_counts;', con=connection)
diag_cd_df

c:\Users\iperez9\.conda\envs\ip_env\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,data_source,year,diag_cd,diag_count
0,trum,2015,S72101K,8
1,trum,2013,9198,4159
2,trum,2022,S61253D,4
3,trum,2022,S86022A,17
4,trum,2013,31535,96
...,...,...,...,...
940172,truc,2020,Y92214,295
940173,truc,2018,H5441,8
940174,truc,2022,M88869,2
940175,truc,2019,T7809XD,1712


We see that we have a lot of claims with diagnosis codes that are not in our reference list. If we take a further look at some of these codes, we see that in our reference list, there is at least one more digit missing from these codes. The missing digits helps specify the diagnosis code.

In [62]:
query = '''
select a.*
from qa_reporting.dw_truv_diag_counts a
left join reference_tables.ref_cms_icd_cm_codes b
on a.diag_cd = cd_value
where b.cd_value is null;
'''

invalid_diag_df = pd.read_sql(query, con=connection)
invalid_diag_df

c:\Users\iperez9\.conda\envs\ip_env\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,data_source,year,diag_cd,diag_count
0,trum,2018,X789X,3
1,truc,2019,S6292X,62
2,truc,2017,T8502X,1
3,trum,2022,Q2112,1207
4,truc,2020,Q2112,1
...,...,...,...,...
8562,truc,2022,W1782X,1
8563,truc,2022,O3504X2,3
8564,truc,2017,T670X,1
8565,truc,2022,O642X,13


Overall, the number of invalid diagnosis codes is neglible compared to the overall number of diagnosis codes in the claim_diag table.

In [63]:
invalid_diag_df.groupby('year')['diag_count'].sum()

year
2014        20
2016        17
2017     75082
2018     68613
2019     83543
2020     52662
2021     65171
2022    534844
Name: diag_count, dtype: int64

In [64]:
diag_comp_df = pd.DataFrame({'overall_diag_count': diag_cd_df.groupby('year')['diag_count'].sum(),
                            'invalid_diag_count': invalid_diag_df.groupby('year')['diag_count'].sum(),
                            'valid_diag_count': diag_cd_df.groupby('year')['diag_count'].sum() - invalid_diag_df.groupby('year')['diag_count'].sum()})
diag_comp_df.loc[diag_comp_df['valid_diag_count'].isna(),'valid_diag_count'] = diag_comp_df.loc[diag_comp_df['valid_diag_count'].isna(),'overall_diag_count']
diag_comp_df['valid_diag_count'] =  diag_comp_df['valid_diag_count'].astype(int)
diag_comp_df['invalid_to_valid_percent'] = 100. * diag_comp_df['invalid_diag_count'] / diag_comp_df['valid_diag_count']
diag_comp_df

,overall_diag_count,invalid_diag_count,valid_diag_count,invalid_to_valid_percent
year,,,,
2011,1000477143,NaN,1000477143,NaN
2012,1042654614,NaN,1042654614,NaN
2013,858067734,NaN,858067734,NaN
2014,975855050,20.0,975855030,0.000002
2015,643804466,NaN,643804466,NaN
2016,688899245,17.0,688899228,0.000002
2017,622576862,75082.0,622501780,0.012061
2018,623119658,68613.0,623051045,0.011012
2019,653272788,83543.0,653189245,0.012790


In [65]:
invalid_diag_df.groupby('year').max()

,data_source,diag_cd,diag_count
year,,,
2014,truc,7070,6
2016,trum,X500X,3
2017,trum,Z28310,6593
2018,trum,Y33XX,4258
2019,trum,Y33XXX,4397
2020,trum,Z28311,3413
2021,trum,Y32XX,4004
2022,trum,Z91A9,54588


## Diagnosis Code Position

In [19]:
query = '''drop table if exists qa_reporting.dw_truv_diag_position;
select data_source, year, table_id_src, diag_position, count(distinct uth_claim_id) as claim_count
into qa_reporting.dw_truv_diag_position
from dw_staging.trum_claim_diag
group by 1,2,3,4;

insert into qa_reporting.dw_truv_diag_position
select data_source, year, table_id_src, diag_position, count(distinct uth_claim_id) as claim_count
from dw_staging.truc_claim_diag
group by 1,2,3,4;
'''
with connection.cursor() as cursor:
    cursor.execute(query)


In [47]:
diag_position_df = pd.read_sql('select * from qa_reporting.dw_truv_diag_position;', con=connection)
diag_position_df.sort_values(['year', 'diag_position'])

c:\Users\iperez9\.conda\envs\ip_env\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,data_source,year,table_id_src,diag_position,claim_count
59,trum,2011,mdcro,1,106352416
71,trum,2011,mdcrs,1,10829887
147,truc,2011,ccaes,1,22319363
159,truc,2011,ccaeo,1,464767402
78,truc,2011,ccaes,2,10268857
...,...,...,...,...,...
130,trum,2022,mdcrs,4,1539385
176,truc,2022,ccaeo,4,44682894
211,truc,2022,ccaes,4,3273070
25,truc,2022,ccaes,P,8128338


In [50]:
diag_position_df['diag_position'].unique()

array(['P', '2', '3', '1', '4'], dtype=object)

Checking if the counts for the diag_position are correct. The higher the diag_position is, the less counts there should be. If we sort the counts of the diag_position by year and assign their order, this value should match with the diag_position value.

In [51]:
diag_position_df['row_rank'] = diag_position_df.sort_values(['data_source', 'year', 'claim_count'], ascending=[True, True, False]).groupby(['data_source', 'year', 'table_id_src']).cumcount()+1

# Primary DX codes should be the most frequent position used in the raw tables, mapping positions based on how frequent they should be
diag_position_df.loc[diag_position_df['table_id_src'].isin(['ccaes', 'mdcrs']), 'diag_position'] = diag_position_df.loc[diag_position_df['table_id_src'].isin(['ccaes', 'mdcrs']), 'diag_position'].map({'P': 1, '1': 2, '2': 3, '3': 4, '4': 5})
diag_position_df['diag_position'] = diag_position_df['diag_position'].astype(int)

diag_position_df['position_check'] = diag_position_df['row_rank'] == diag_position_df['diag_position']
diag_position_df[~diag_position_df['position_check']]

,data_source,year,table_id_src,diag_position,claim_count,row_rank,position_check


## ICD Version

In [66]:
query = '''drop table if exists qa_reporting.dw_truv_icd_version_count;
select data_source, year, icd_version, count(distinct uth_claim_id) as claim_count
into qa_reporting.dw_truv_icd_version_count
from dw_staging.trum_claim_diag
group by 1,2,3;

insert into qa_reporting.dw_truv_icd_version_count
select data_source, year, icd_version, count(distinct uth_claim_id) as claim_count
from dw_staging.truc_claim_diag
group by 1,2,3;
'''
with connection.cursor() as cursor:
    cursor.execute(query)

In [70]:
icd_version_df = pd.read_sql('select * from qa_reporting.dw_truv_icd_version_count;', con=connection)
icd_version_df.sort_values(['data_source', 'year', 'icd_version'])

c:\Users\iperez9\.conda\envs\ip_env\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,data_source,year,icd_version,claim_count
61,truc,2011,0,1456
34,truc,2011,9,4184
37,truc,2011,None,486715834
47,truc,2012,0,1790
29,truc,2012,9,5283
...,...,...,...,...
42,trum,2021,9,1048
65,trum,2021,None,4645
36,trum,2022,0,42626513
59,trum,2022,9,33


In [71]:
icd_version_df[icd_version_df['data_source'] == 'truc'].sort_values(['year', 'icd_version'])

,data_source,year,icd_version,claim_count
61,truc,2011,0,1456
34,truc,2011,9,4184
37,truc,2011,None,486715834
47,truc,2012,0,1790
29,truc,2012,9,5283
58,truc,2012,None,496302656
68,truc,2013,0,3442
52,truc,2013,9,10258
69,truc,2013,None,390003953
62,truc,2014,0,9867


In [72]:
icd_version_df[icd_version_df['data_source'] == 'trum'].sort_values(['year', 'icd_version'])

,data_source,year,icd_version,claim_count
26,trum,2011,0,347
4,trum,2011,9,970
35,trum,2011,None,116980600
8,trum,2012,0,379
9,trum,2012,9,1107
12,trum,2012,None,109005110
16,trum,2013,0,836
28,trum,2013,9,2865
70,trum,2013,None,96481406
66,trum,2014,0,2957
